# Class 3 - Linear regression

## The model

Linear models are widely used and form the foundation of many advanced non-linear techniques such as support vector machines and neural networks. You propably have seen an image depicting linear regression already:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, model_selection, linear_model, metrics

In [ ]:
n = 30
beta = 3

x = 4*np.random.random(n)
eps = 2*np.random.random(n)-1
y = x*beta + eps

plt.plot(x,y,'ob',[0,4],[0,4*beta],'r')
plt.show()

Consider $\mathcal{X} = ℝ^d$ and $\mathcal{Y} = ℝ$. For linear regression, we assume that the data are of the form $$y \approx \sum_{i=1}^d β_i x^{(i)} + β_0.$$ Hence, we choose the hypothesis as the class of affine functions: $$\{x\mapsto \langleβ,x\rangle + β_0 \;\vert\; β_0,…,β_d ∈ℝ\}.$$

Usually, it is more convenient to incorporate the bias $β_0$ into the variables $β$ as well. To do so, denote $x_0 = 1$ and consider instead $\mathcal{X} = ℝ^{d+1}$ with $x = (1,x^{(i)},…,x^{(i)})$. We will do so implicitly and hence will work with the hypothesis class of linear functions instead:
$$\mathcal{F} = \{x\mapsto \langleβ,x\rangle \;\vert\; β∈ℝ^{d+1}\}$$

Which loss function should we choose? It is common to consider one of the following two:
- squared error (OLS) $L(y,y') = (y−y')^2$
- absolute value loss function $L(y,y') = |y−y'|$

Don't use whatever loss function comes to your mind if there is no need for it! Stay with the well-known choices.

Denote $X = (x_i^{(j)})_{i,j}$ the matrix with rows given by $x_1,…,x_n$. Then, we and up with the following problem: Find $\operatorname{argmin}_{h∈\mathcal{F}} \hat{R}(h)$ or more explicitely
* $\operatorname{argmin}\{ \frac{1}{n} \sum_{i=1}^n (y_i − \langleβ,x_i\rangle)^2 \;\vert\; β∈ℝ^d\} = \operatorname{argmin}\frac{1}{n} \|Xβ−y\|_2^2$, resp.
* $\operatorname{argmin}\{ \frac{1}{n} \sum_{i=1}^n |y_i − \langleβ,x_i\rangle| \;\vert\; β∈ℝ^d\} = \operatorname{argmin}\frac{1}{n} \|Xβ−y\|_1$

Advantages:
- Easy to understand model
- Scale well on large datasets
- Work well with sparse data
- Quite stable for small changes in the input data
- Rarely overfits (low variance)


Disadvantages:
- Linear model might not represent the truth
- Tend to underfit (high bias)
- Cannot treat well strongly correlated features e.g. $x_1 = −x_2$

## The algorithm - by foot

We will present the algorithms on the Boston house price data set from scikit-learn. Throughout the whole part we will use the squared error.

In [ ]:
boston = datasets.load_boston()

print(boston.DESCR)

In [ ]:
print(boston.data.shape)
print(boston.target.shape)

In [ ]:
x, y = boston.data, boston.target
y = y.reshape((y.shape[0], 1))

### Preparation of data

In general it is a bad idea to adhoc start with your machine learning algorithm. Instead we should first take care of the data.

#### Shuffling

Usually, data are not generated from an i.i.d. source as we always assume. Instead, the samples might be time-dependent without us noticing or are correlated in some other way among each other. Hence, it is always a good idea to shuffle them before usage.

In [ ]:
def shuffle(x, y):
    z = np.hstack((x, y))
    np.random.shuffle(z)
    return np.hsplit(z, [x.shape[1]])

x, y = shuffle(x, y)

#### Normalization

There is a need to normalize all features to let them live on the same scale, called feature scaling. In the special case of linear regression this is usually not needed. Though, some implementations of linear regression might still profit from data being not too large. Other algorithms cannot be run without normalization. Normalization usually means
$$x_{new} = \frac{x_{old}−x_{mean}}{x_{sd}}$$
and
$$y_{new} = y_{old} − y_{mean}.$$
Make sure that as soon as you normalized your data, you have to do the reverse once you want to interpret your results! For sake of an easier presentation, we omit normalization here.

#### Train and test splitting

To obtain a result of how good our predictions will be we cannot plainly use the empirical error obtained. This error is biased as we trained the model to recreate the data presented. Instead, one typically calculates the empirical error on new, previously unseen data to obtain a better estimate on how good the model describes reality. This step is called testing. As a rule of thumb around 20% of your total data should be hold back for testing in the end.

In [ ]:
def splitting(x, y, test_size=0.2):
    n = x.shape[0]
    train_size = int(n * (1 - test_size))
    return x[:train_size, ], x[train_size:, ], y[:train_size, ], y[train_size:, ]

x_train, x_test, y_train, y_test = splitting(x, y)

Before starting, let's implement a function yielding the empirical error.

In [ ]:
def linear_regression_empirical_risk(x, y, b):
    return 1 / x.shape[0] * (np.linalg.norm(x.dot(b) - y) ** 2)

### (Batch) Gradient descent

The function $β \mapsto \frac{1}{n} \|Xβ−y\|_2^2$ is convex. Hence, we can apply a standard gradient descent method to find its minimum. The gradient is given by
$$\nabla_β \frac{1}{n} \|Xβ−y\|_2^2 = \frac{2}{n}(X^t X β − X^t y).$$
Fix some step length/learning rate $α∈ℝ_+$. Then, we iterate the step
$$ β_{new} = β_{old} − \frac{2α}{n}(X^t X β − X^t y)$$
until the algorithm converges.

What is the role of $α$? In theory, one can choose $α$ depending on the iteration step. A typical choice is $α = \frac{(\nabla_β \frac{1}{n} \|Xβ−y\|_2^2)^t (\nabla_β \frac{1}{n} \|Xβ−y\|_2^2)}{(\nabla_β \frac{1}{n} \|Xβ−y\|_2^2)^t x^t x (\nabla_β \frac{1}{n} \|Xβ−y\|_2^2)}$ (Gauss-Newton algorithm). Another approach is to choose alpha in the beginning by hand. Then, one calls $α$ a hyperparameter (a parameter in your algorithm which is not learned). How to treat hyperparameters will be the content of a future class.

In [ ]:
def linear_regression_gradient(x, y, alpha):
    n, d = x.shape
    result = np.zeros((d, 1))
    cost_new = linear_regression_empirical_risk(x, y, result)
    cost_old = 0
    i = 0
    while (cost_new - cost_old) ** 2 > 10 ** (-20):
        gradient = (np.transpose(x)).dot(x.dot(result) - y)
        alpha = (np.transpose(gradient).dot(gradient)) / (
            (np.transpose(gradient).dot(np.transpose(x))).dot(x.dot(gradient)))
        result -= (2 * alpha / n) * gradient
        cost_new, cost_old = linear_regression_empirical_risk(x, y, result), cost_new
        i += 1
    print("Iterations: {}".format(i))
    return result

beta_grad = linear_regression_gradient(x_train, y_train, 0.1)

### The normal equation

Actually, a closed form solution for our minimization problem
$$\operatorname{argmin}_β \frac{1}{n} \|Xβ−y\|_2^2 = \frac{1}{n}(β^t X^t X β − 2β^t X^t y + y^t y)$$
exists. The gradient is given by
$$\nabla_β \frac{1}{n} \|Xβ−y\|_2^2 = \frac{2}{n}(X^t X β − X^t y).$$
This immediately yields
$$β = (X^t X)^{−1} X^t y.$$
In general, $X^t X$ must not be invertible. This is particularly the case if the number of features is high compared to the sample size. In such a case the equation for $β$ admits several solutions. One of them can be found by replacing the inverse matrix by a generalized inverse/pseudoinverse $X^+$. This leads to the solution
$$β = X^+ y.$$

In [ ]:
def linear_regression_normal(x, y):
    result = np.linalg.pinv(x).dot(y)
    return result

beta_normal = linear_regression_normal(x_train, y_train)

## The algorithm - by scikit-learn

By using scikit-learn, life gets so much easier...

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, shuffle=False, test_size=0.2)
lr = linear_model.LinearRegression(fit_intercept=False, normalize=False).fit(x_train, y_train)

beta_sklearn = np.transpose(lr.coef_)

Some comments:
* scikit-learn treats machine learning algorithms as classes. For example, .fit is a method of the class LinearRegression.
* All derived information are stored in variables with a trailing underscore by scikit-learn.

Scikit-learn has also implemented tools to predict data and calculate the mean squared error or mean absolute error:

In [ ]:
y_predict = lr.predict(x_test)
metrics.mean_squared_error(y_test,y_predict)

## Testing

Let's see how our different algorithms perform.

In [ ]:
print("----------Normal----------")
print("My estimated beta: {}".format(beta_normal))
print("Empirical error on training set (normal): {:.2f}".format(
    linear_regression_empirical_risk(x_train, y_train, beta_normal)))
print("Empirical error on test set (normal): {:.2f}".format(
    linear_regression_empirical_risk(x_test, y_test, beta_normal)))

print("----------Gradient----------")
print("My estimated beta: {}".format(beta_grad))
print("Empirical error on training set (grad): {:.2f}".format(
    linear_regression_empirical_risk(x_train, y_train, beta_grad)))
print("Empirical error on test set (grad): {:.2f}".format(linear_regression_empirical_risk(x_test, y_test, beta_grad)))

print("----------Scikit-Learn----------")
print("lr.coef_: {}".format(beta_sklearn))
print("Empirical error on training set (sklearn): {:.2f}".format(
    linear_regression_empirical_risk(x_train, y_train, beta_sklearn)))
print("Empirical error on test set (sklearn): {:.2f}".format(
    linear_regression_empirical_risk(x_test, y_test, beta_sklearn)))

Scikit-learn also provides an additional value to get an idea of the performance of our machine learning algorithm, the so-called score $R^2$ (also known as explained variance or coefficent of determination). It is given by
$$R^2 = (1-\frac{R(β)}{\operatorname{Var}(y)}).$$
As the name suggests it provides the proportion of the variance in the dependent variable that is predictable from the independent variable(s).

In [ ]:
print("Training set score: {:.2f}".format(lr.score(x_train, y_train)))
print("Test set score: {:.2f}".format(lr.score(x_test, y_test)))

## Discussion

Is this learning?
* No iterative learning steps in normal equation does not mean that the algorithm is not learning.
* Linear regression profits a lot from its easy form. This makes the algorithm explainable.

Normal equation vs gradient descent
* Calculating inverse matrix is difficult for many features. Updating rule stays easy.
* Learning rate has to be chosen well for gradient descent to work. So the learning algorithm gets more difficult. --> See also Class 4

Theoretical bounds
* Consider a data set $(x^{(i)},y_i)_{i=1,…,n}$ following the model $y = x^t β + ε$ for i.i.d. noise $ε$ with mean 0 and variance $σ^2$.
* The training error satisfies $\mathbb{E}[R(β)] = σ^2 (1−\frac{d+1}{n_{test}})$.
* The test error satisfies $\mathbb{E}[R(β)] = σ^2 (1+\frac{d+1}{n_{train}})$.
* This shows that the model is indeed learning some information from the noise as well.


## Practice yourself!

Play around with the code yourself! Possible ideas that might lead you to interesting observations are:
1. Compare all algorithms for the squarred error.
2. Implement the normalization. How do the results of your algorithms change?
3. Extend the algorithms to allow for affine hypotheses. How do the results of your algorithms change?
2. Implement the algorithm for the absolute value loss function. Compare the estimator for the absolute value loss function with the one for the squared error.
4. Try your algorithms on the new dataset diabetes from scikit-learn.
5. Try your algorithm on a data set you generated yourself: Fix some $β∈ℝ^d$ with possibly some parameters being (close to) zero. Generate some samples via $y = \sum_{i=1}^d β_i x^{(i)} + ε$ with some i.i.d. noise $ε$.
6. Plot the training error and the test error for your model trained on smaller sized data sets of size $m\le n$ for different $m$. Can you see how your model is “learning”?